In [ ]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
DEVICE = 'cuda'
NUM_CLASSES= 7
BATCH_SIZE = 128
LR=1e-4
MOMENTUM= 0.9
WEIGHT_DECAY = 5e-5

NUM_EPOCHS=30
STEP_SIZE=20
GAMMA= 0.1

LOG_FREQUENCY=10
ALPHA=0.5
cross_val_domain=True




In [ ]:
alexNet_transform = transforms.Compose([
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                            ])

In [ ]:
from torchvision.datasets import VisionDataset, ImageFolder
from PIL import Image
import os
import os.path
import sys
import shutil
import numpy as np

# Clone github repository with data
if not os.path.isdir('./PACS'):
  !git clone https://github.com/MachineLearning2020/Homework3-PACS
  !mv 'Homework3-PACS' 'PACS'
  
  

DATA_DIR = 'PACS/PACS'


def pil_loader(path):
  
  #path=path.rstrip("\n")
  with open(path, 'rb') as f:
    img=Image.open(f)
    return img.convert('RGB')




class Pacs(VisionDataset):
    def __init__(self, root, split='', transform=None, target_transform=None):
        
        
        self.split = split
        self.root = os.path.join(root,self.split)
        #print(self.root)
        self.transform = transform
        

        self.dataset= []

        self.dataset = ImageFolder(self.root, transform=transform, loader=pil_loader)
        print(self.dataset[2])
        print(len(self.dataset))

    def __getitem__(self, index):
      
      return self.dataset[index]

    def __len__(self):
      return len(self.dataset)
          

art_painting=Pacs(DATA_DIR, 'art_painting', transform=alexNet_transform)
cartoon=Pacs(DATA_DIR, 'cartoon', transform=alexNet_transform)
photo=Pacs(DATA_DIR, 'photo', transform=alexNet_transform)
sketch=Pacs(DATA_DIR, 'sketch', transform=alexNet_transform)

#photoValidation= photo.dataset.samples[:, len(photo)-200:len(photo)]

#print(sketch.__getitem__(2))

print(".sketch",sketch.__len__)
print("phptp",photo.__len__)
print("cartoon",cartoon.__len__)
print("art",art_painting.__len__)

def imgshow(img, mean=None, std=None):
	if mean == None or std == None:
		# use (0.5 0.5 0.5) (0.5 0.5 0.5) as mean and std
		img = img / 2 + 0.5     # unnormalize
		npimg = img.numpy()
		plt.imshow(np.transpose(npimg, (1, 2, 0)))
		plt.show()
		# raise RuntimeError("You should pass mean and std to 'imgshow' method")
	else : 
		# use custom mean and std computed on the images
		mean = np.array(mean)
		std = np.array(std)
		for i in range(3): 
			img[i] = img[i]*std[i] + mean[i] # unnormalize

		npimg = img.numpy()
		plt.imshow(np.transpose(npimg, (1, 2, 0)))
		plt.show()
	return


class0=0
class1=0
class2=0
class3=0
class4=0
class5=0
class6=0

leng= len(photo)
for i in range(leng):
  if sketch.dataset.targets[i] == 0:
    class0= class0+1
    imgshow(art_painting.dataset[i][0])
  elif photo.dataset.targets[i] == 1:
    class1= class1+1
  elif photo.dataset.targets[i] == 2:
     class2=class2 + 1
  elif photo.dataset.targets[i] == 3:
      class3= class3+1 
  elif photo.dataset.targets[i] == 4:
     class4= class4+1
  elif photo.dataset.targets[i] == 5:
     class5 = class5+1
  elif photo.dataset.targets[i] == 6:
     class6 = class6+1


print(class0, class1, class2, class3, class4, class5, class6)
#print(photo.dataset.class_to_idx)
print(class0 + class1 + class2 + class3 + class4 + class5 + class6)
'''

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1)

train_indexes = [] # split the indices for your train split
val_indexes = [] #split the indices for your val split



for train, val in sss.split(photo.dataset.samples, photo.dataset.targets):
  print("train-----",train)
  print("test-----",val)
  train_indexes=train
  val_indexes=val

photo_val=Subset(photo, val_indexes)
photo= Subset(photo, train_indexes)

print(len(photo))
print(len(photo_val))
'''






In [ ]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
from torchvision import utils
import matplotlib.pyplot as plt
#print(cartoon[0])

cartoon_dataloader = DataLoader(cartoon, batch_size=1, shuffle=True, num_workers=4, drop_last=True)
art_painting_dataloader = DataLoader(art_painting, batch_size=1, shuffle=True, num_workers=4, drop_last=True)
photo_dataloader = DataLoader(photo, batch_size=1, shuffle=True, num_workers=4, drop_last=True)
sketch_dataloader = DataLoader(sketch, batch_size=1, shuffle=True, num_workers=4, drop_last=True)

#photo_val_dataloader = DataLoader(photo_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=False)
#val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
#train_val_dataloader =  DataLoader(train_val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


#test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

xb, yb = next(iter(cartoon_dataloader))
out = torchvision.utils.make_grid(xb)
plt.imshow(out.numpy().transpose((1, 2, 0)))

print(photo_dataloader.dataset[0])



In [ ]:
import torch.nn as nn
from torch.autograd import Function

class ReverseLayerF(Function):
    # Forwards identity
    # Sends backward reversed gradients
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)
        
    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


        

In [ ]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url



__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
            
        )

        self.domainClassifier1 = nn.Sequential(
          nn.Dropout(),
          nn.Linear(256 * 6 * 6, 4096),
          nn.ReLU(inplace=True),
          nn.Dropout(),
          nn.Linear(4096, 4096),
          nn.ReLU(inplace=True),
          nn.Linear(4096, 2)
          
        )

        self.domainClassifier = nn.Sequential(
          nn.Dropout(),
          nn.Linear(256 * 6 * 6, 4096),
          nn.ReLU(inplace=True),
          nn.Dropout(),
          nn.Linear(4096, 4096),
          nn.ReLU(inplace=True),
          nn.Linear(4096, 2)
          
        )

    def conv (self, x):
      x = self.features(x)
      return x

    def forward(self, x, alpha=None):
        #print(x.size())
        features = self.features(x)
        features = self.avgpool(features)

        #print("before view avgPool",features.size()) #([1, 256, 6, 6])
        features = features.view(features.size(0), -1) 
        #print("after view",features.size()) #([1, 9216])

        if alpha is not None:
          reverse_feature = ReverseLayerF.apply(features, alpha)
          #print("rf size",reverse_feature.size())  #([1, 9216])   
          discriminator_output = self.domainClassifier1(reverse_feature)
          return discriminator_output
        else:
          x = self.classifier(features)
          return x


def alexnet(pretrained=False, progress=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict, strict=False)
    return model


net=alexnet(pretrained=True)
net.classifier[6] = nn.Linear(4096, 7)

weights=net.classifier[1].weight.data
bias= net.classifier[1].bias.data


net.domainClassifier1[1].weight.data = weights
net.domainClassifier1[1].bias.data = bias
print(net)
'''
print(net.features(photo.__getitem__(0)[0].unsqueeze(0)).size())
#print(net.features(photo.__getitem__(0)[0].unsqueeze(0)))
img= net.features(photo.__getitem__(0)[0].unsqueeze(0))
img = net.avgpool(img)
print("--------------",img.size())
img=net.forward(img, alpha=1)
print(img.size())
'''
img=net.conv(photo.__getitem__(0)[0].unsqueeze(0))
print(img.size())

In [ ]:
criterion = nn.CrossEntropyLoss()

parameters_to_optimize = net.parameters()

optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [ ]:
# train on photo and test on art painting without adaptation


  
accuracy_train=[]

net = net.to(DEVICE)
cudnn.benchmark


current_step = 0
loss_train=[]
accuracy_test=[]

for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  running_corrects=0
  for images, labels in photo_dataloader:
    images=images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train()

    optimizer.zero_grad()

    outputs = net(images, alpha=None)
    #print("outputssize", outputs.size())
    

    loss= criterion(outputs, labels)

    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    loss.backward()
    optimizer.step()
    current_step += 1
    scheduler.step()


if cross_val_domain:
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in cartoon_dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(cartoon))

  print('Test Accuracy: {}'.format(accuracy))


  running_corrects = 0
  for images, labels in sketch_dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy_2 = running_corrects / float(len(sketch))

  print('Test Accuracy: {}'.format(accuracy_2))
    
print("average accuracy: ", (accuracy + accuracy_2)/2)
 


In [ ]:
print(accuracy)
print(len(loss_train))
plt.plot(loss_train, label='Training Loss')

plt.legend(loc='upper right')
plt.ylabel('Loss')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

In [ ]:
plt.plot(loss1, label="LR=0.01")
plt.plot(loss2, label="LR=0.001")
plt.plot(loss3, label="LR=0.0001")
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.ylim([-0.001,0.02])
plt.title('Training Loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

In [ ]:
# domain adaptation with DANN
net = net.to(DEVICE)
cudnn.benchmark
loss_1=[]
loss_2=[]
loss_3=[]


current_step=0
for epoch in range(NUM_EPOCHS):
   print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

   running_corrects=0

   #len_dataloader = min(len(photo_dataloader), len(art_painting_dataloader))
   #source = iter(photo_dataloader)
   #target = iter(art_painting_dataloader)

   for imgs_target,labels_target in sketch_dataloader:#range del target
    
     data_source = next(iter(photo_dataloader)) #prendo label del target
     #target_source = target.next()

     imgs_source, labels_source = data_source
     #art_batch, art_labels = target_source

     #print("PHOTO BATCH size",photo_batch.size())
     #print("photo_batch labels size", photo_labels.size())

     imgs_source = imgs_source.to(DEVICE)
     labels_source = labels_source.to(DEVICE)
     imgs_target = imgs_target.to(DEVICE)
     labels_target = labels_target.to(DEVICE)

     net.train()

     optimizer.zero_grad()

     outputs_classifier = net(imgs_source, alpha=None)  #tensor of 128 rows and 7 columns, with the proability for each class
     #print("putputeclass",outputs_classifier.size())

     loss1 = criterion(outputs_classifier, labels_source)
     #print("loss",loss1)
     loss1.backward()
     #print("oooooooooo")
     #print(photo_batch[0])

     outputs_source_domain = net(imgs_source, alpha=ALPHA)
     source_domain_labels = torch.zeros(len(outputs_source_domain))
     source_domain_labels=source_domain_labels.long()
     source_domain_labels=source_domain_labels.to(DEVICE)
     
     #print("size source domain",outputs_source_domain.size())
     #print("size source domain",source_domain_labels.size())

     loss2 = criterion(outputs_source_domain, source_domain_labels)
     loss2.backward()
    

     outputs_targets_domain = net(imgs_target, alpha=ALPHA)
     target_domain_labels = torch.ones(len(outputs_targets_domain))
     target_domain_labels=target_domain_labels.long()
     target_domain_labels=target_domain_labels.to(DEVICE)
     loss3 = criterion(outputs_targets_domain, target_domain_labels)
     loss3.backward()

     if current_step % LOG_FREQUENCY == 0:
       print('Step {}, Loss1 {}, Loss2 {}, Loss3 {}'.format(current_step, loss1.item(), loss2.item(), loss3.item()))

     optimizer.step() # update weights based on accumulated gradients

     current_step += 1
   loss_1.append(loss1)
   loss_2.append(loss2)
   loss_3.append(loss3)
   scheduler.step()
   scheduler.step()

net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in sketch_dataloader:
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

   # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(sketch))

print('Test Accuracy: {}'.format(accuracy))
     

print(loss_1)
print(loss_2)
print(loss_3)



In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(sketch_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(sketch))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 15/15 [00:11<00:00,  1.28it/s]

Test Accuracy: 0.1598371086790532


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

np.set_printoptions(precision=2)
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
X_test= art_painting.dataset.samples
y_test=art_painting.dataset.targets
for title, normalize in titles_options:
    disp = plot_confusion_matrix(net, X_test, y_test,
                                 display_labels="33",
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)